
> **Model Optimization:** in some cases we reduce the layers and faster operations, less fully connected layers and more.. \
in most cases this strategy will work but in this way we sacrifice the accuracy over speed and in most cases we don't want to do that.

> **Quantization:** a technique where we reduce precision spaces and after the floating numbers from a regular floating. we are converting them into 8 bit numbers in most cases.. \
in most cases these tichnique won't influence our model accuracy and there are two ways to perform quantization of the model,, 1) pre-training quantization and another one is 2) bost training quatization. \
just to count more there is a parameter count reduction as an optimization strategy and finally here is a link to excellent blog post about the model quantization process.

**Putting a trained model on a mobile devices:** https://heartbeat.fritz.ai/neural-networks-on-mobile-devices-with-tensorflow-lite-a-tutorial-85b41f53230c \

**TensorFlow Lite tutorial and guide:** https://www.tensorflow.org/lite/guide

**TensorFlow Lite pre-trained models with applications:**  https://www.tensorflow.org/lite/examples/

**Putting a trained model on a mobile device:** https://medium.com/@rdeep/tensorflow-lite-tutorial-easy-implementation-in-android-145443ec3775



### Import Project dependencies

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

'2.4.1'

### Dataset preprocessing
#### Loading the dataset 

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


#### Image Normalization

In [4]:
X_train = X_train / 255.
X_test = X_test / 255.

#### Reshaping the dataset

In [5]:
X_train.shape

(60000, 28, 28)

> for our artichicture that accept only vectors not metrixes for that we need to reshape the data

In [6]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

### Building a model
#### Defining the model

In [8]:
model = tf.keras.models.Sequential()

#### Building the model

In [9]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

#### Compiling the model

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


#### Training the model

In [12]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 2ms/step - loss: 0.6873 - sparse_categorical_accuracy: 0.7609
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4063 - sparse_categorical_accuracy: 0.8518
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3671 - sparse_categorical_accuracy: 0.8657
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3410 - sparse_categorical_accuracy: 0.8729
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3305 - sparse_categorical_accuracy: 0.8789


### Evaluating the model

In [13]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

313/313 [==============================] - 1s 2ms/step - loss: 0.3586 - sparse_categorical_accuracy: 0.8685
Test accuracy: 0.8684999942779541


### Converting the model into TensorFlow Lite version
#### Saving the model

In [16]:

model_name = 'fashion_mobile_model.h5'
!mkdir -p saved_model
model.save(f'saved_model/{model_name}')

#### Creating the TFLite Converter

In [23]:
%pwd

'/content'

In [33]:
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(model_name)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


#### Converting the model

In [34]:
# model = tf.keras.models.load_model( 'fashion_mobile_model.h5' )

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpxo5xife5/assets
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /tmp/tmpxo5xife5/variables/variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: dense_input
INFO:tensorflow: tensor name: serving_default_dense_input:0, shape: (-1, 784), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: dense_1
INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 10), type: DT_FLOAT
INFO:tensorflow:Restoring parameters from /tmp/tmpxo5xife5/variables/variabl

#### Saving the tflite model 

In [35]:
# First thing we need to do is to open our model
with open('tf_model.tflite', 'wb') as f:
  f.write(tflite_model)